# Installing the Opik Library

In [ ]:
!pip install opik


#Importing Necessary Libraries and Configuring Opik





In [ ]:
import opik
from opik import Opik, track
from opik.evaluation import evaluate
from opik.evaluation.metrics import (Hallucination, AnswerRelevance)
import pandas as pd
import os

opik.configure(use_local=False)

#Setting the OpenAI API Key

In [ ]:
open_api_key = os.environ.get('OPENAI_API_KEY')

#Loading the CSV File into a DataFrame
###In this cell, we load the CSV file into a pandas DataFrame. The CSV contains the query, LLM response, and retrieved passage, which will be used for evaluating the performance of the language model.



In [ ]:
file_path = "Enter csv file path here"
df = pd.read_csv(file_path)
df

#Evaluating the LLM Responses
In this cell, we process each row of the CSV file and evaluate the LLM response using the Hallucination and AnswerRelevance metrics. The results are saved in batches to avoid exceeding the rate limit, and progress is saved periodically.

In [ ]:
import time

results = []
metrics = [Hallucination(), AnswerRelevance()]
batch_size = 3  # Process in batches to reduce the chance of exceeding the rate limit

for i in range(len(df)):
    try:
        row = df.iloc[i]
        scores = {"query": row['Question'], "answer": row['Response']}

        for metric in metrics:
            try:
                result = metric.score(
                    input=row['Question'],
                    output=row['Response'],
                    context=row['passage'],
                )
                if isinstance(metric, Hallucination):
                    scores["hallucination_score"] = result.value
                    scores["hallucination_reason"] = result.reason
                elif isinstance(metric, AnswerRelevance):
                    scores["answer_relevance_score"] = result.value
                    scores["answer_relevance_reason"] = result.reason
            except Exception as e:
                print(f"Rate limit reached: {e}. Retrying in 15 seconds...")
                time.sleep(15)  # Wait and retry
                continue

        results.append(scores)

        # Save progress every batch_size rows
        if i % batch_size == 0:
            pd.DataFrame(results).to_excel("intermediate_results.xlsx", index=False)
            print(f"Progress saved at iteration {i}")

    except Exception as e:
        print(f"Error at row {i}: {e}. Skipping to the next row...")

# Save final results
result_df = pd.DataFrame(results)
result_df.to_excel("final_results.xlsx", index=False)
print("Processing completed. Results saved to final_results.xlsx.")